In [9]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from FPN import Features, FPNetwork , classificationhead , bboxhead
from Loss import Lossfunction
from datasets import load_dataset

from dataset_convert import AnchorGenerator, FaceDetectionDataset
device = torch.device("mps")

In [10]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model = model.features.to(device)

Using cache found in /Users/vipulagarwal/.cache/torch/hub/pytorch_vision_v0.10.0
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
dataset = load_dataset("CUHK-CSE/wider_face")
train_dataset = dataset['train'].with_format("torch")
val_dataset = dataset['validation'].with_format("torch")

In [4]:
extractor = Features(model,['3','6', '13','18'])
topdown = FPNetwork(out_channels=256)
classifier = classificationhead(channels=256, num_anchors= 12, num_of_classes= 1)
bboxregression = bboxhead(channels= 256 , num_anchors= 12)
loss =Lossfunction()
anchors = AnchorGenerator()
data = FaceDetectionDataset(train_dataset,anchors)
vali_data = FaceDetectionDataset(val_dataset,anchors)

In [5]:
# hyperparametrs
batch_size = 1
epochs = 1
learning_rate = 1e-3

In [6]:
training_data = DataLoader(data, batch_size=batch_size, shuffle = True)
validation_data = DataLoader(vali_data, batch_size=batch_size, shuffle = True)

In [7]:

def forward(p):
    features = extractor.extract(p)
    newfeatures = topdown(features)
    output = {}
    for key in list(newfeatures.keys()):
        temp = {}
        temp["bbox"] = bboxregression(newfeatures[key])
        temp["cls"] = classifier(newfeatures[key])
        output[key] = temp
        
    return output
check = False

def train(epochs:int,training_data , validation_data=None):
    extractor.train()
    topdown.train()
    classifier.train()
    bboxregression.train()
    optimizer = optim.Adam(list(extractor.parameters())+
                           list(topdown.parameters())+
                           list(classifier.parameters())+
                           list(bboxregression.parameters()), lr=learning_rate)
    loSS = {}
    for i in range (0,epochs):
        train_loss= 0
        for key, (image,bbox) in enumerate(training_data):
            epoch_loss = {}
            model_pred = forward(image)
            optimizer.zero_grad()
            ll = loss(model_pred, bbox)
            train_loss+=ll
            if (key % 10 == 0):
                epoch_loss[key] = train_loss/10
                print(f"The avg loss for {key}th data is {train_loss/10}")
                train_loss = 0    
            ll.backward()
            optimizer.step()
            print("suicide")
            check = True
            
            break
        if check:
            break
        loSS[i] = epoch_loss
        
    return loSS

In [8]:
total_params=sum(p.numel() for p in extractor.parameters())
total_params+= sum(p.numel() for p in topdown.parameters())
total_params+= sum(p.numel() for p in classifier.parameters())
total_params+= sum(p.numel() for p in bboxregression.parameters())

print(total_params)

7452536


In [8]:
train(epochs=epochs,training_data=training_data)

RuntimeError: MPS backend out of memory (MPS allocated: 18.66 GB, other allocations: 61.09 MB, max allowed: 20.40 GB). Tried to allocate 5.49 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [45]:
type(training_data)

torch.utils.data.dataloader.DataLoader